# Use Globus Compute/Flows with WPS

## Setup Globus Compute Client

In [1]:
import time
import json
import datetime
import globus_sdk
import os
from dotenv import load_dotenv

from globus_sdk import TimerJob
from globus_compute_sdk import Executor, Client
from globus_sdk.experimental.globus_app import UserApp
from globus_sdk.utils import slash_join

import xarray as xr

## Configure the Endpoint and Submit the Function

You should have already followed the instructions in the [compute environment](../compute_environment/README.md) section, having:
- a globus-compute endpoint
- a registered function on the globus-compute endpoint


### Load in Your Compute Endpoint and Function ID
If you followed until the last step, you should use environment file (.env) to feed this into your notebook.

In [9]:
load_dotenv()
compute_endpoint= os.getenv("ENDPOINT_UUID")
function_id = os.getenv("FUNCTION_UUID")
odir = os.getenv("ODIR")

### Prepare your Executor
Once you have your variables, you can instantiate your executor, which is what you will pass your function + arguments to!

In [10]:
gce = Executor(endpoint_id=compute_endpoint)

### Run the Function!

Now that we have our executor ready, we can prepare our arguments and feed that into `submit_to_registered_function()` method!

In [11]:
# Prepare payload for ESGF wps
esgf_wps_data = {
    "node": "DKRZ",
    "start_date":"1990-01-01",
    "end_date":"2000-01-01",
    "lat_min":0,
    "lat_max":35,
    "lon_min":65,
    "lon_max":100,
    "average_frequency":"year",
    "experiment_id":["historical"],
    "variable_id":["tas"],
    "member_id":["r1i1p1f1"],
    "table_id":["Amon"],
    "institution_id":["MIROC"],
    "odir":odir
}

# Start the task
future = gce.submit_to_registered_function(function_id, kwargs=esgf_wps_data)

### Check on the output
The output of the function is a path to the file(s)! We can then read that in with [xarray](https://xarray.dev/) since it is a netCDF file.

In [12]:
future.result()

['/Users/mgrover/git_repos/esgf-crocus-globus-flows/data/tas_Amon_MIROC6_historical_r1i1p1f1_gn_19900101-19990101_avg-year.nc']

In [13]:
ds = xr.open_dataset(future.result()[0])
ds

<xarray.Dataset> Size: 34kB
Dimensions:    (time: 10, lat: 25, bnds: 2, lon: 25)
Coordinates:
  * lat        (lat) float64 200B 0.7004 2.101 3.502 4.903 ... 31.52 32.92 34.32
  * lon        (lon) float64 200B 66.09 67.5 68.91 70.31 ... 97.03 98.44 99.84
    height     float64 8B ...
  * time       (time) datetime64[ns] 80B 1990-01-01 1991-01-01 ... 1999-01-01
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 4kB ...
    lon_bnds   (time, lon, bnds) float64 4kB ...
    tas        (time, lat, lon) float32 25kB ...
    time_bnds  (time, bnds) datetime64[ns] 160B ...
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2018-11-30T16:12:41Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.3.2
    tracking_id:            hdl:21.14100/8eafad89-f8c0-4266-b1eb-b5c103293111

## Running a Globus Flow
Now that we have our compute job, we can add in a transfer step, fully automating with a full "flow" for more on Globus-Flows, read more here!

https://docs.globus.org/api/flows/

### Configure Your Flow Client
The first step is establishing your flow client - which requires its own client ID!

#### Retrieve Your Client ID for Flows

1. Go to the developer's settings on globus.org

https://app.globus.org/settings/developers

![Globus Developers Settings](../images/globus-settings-developers.png)

2. Create a new project (or use an existing one if you have already)

If you have not already done so, select "Register a thick client..." and create new project as shown below.
![Register a Thick Client](../images/create-new-project.png)

3.Create a thick client for your new project

Go to your new project, and select add an app.
![New App](../images/add-new-app.png)

Then select "Register a thick client..."
![New Client](../images/register-thick-client.png)

4. Extract your thick client ID from your newly create application by clicking on the application
![Thick client ID](../images/thick-client-id.png)

#### Feed Your Client ID Into the Flows Client
Now that you have your ID, add it the variable `CLIENT_ID` below. 

In [3]:
CLIENT_ID = "3ec14a98-4bb5-492d-b71a-697960482d57"
my_app = UserApp("esgf-wps-app", client_id=CLIENT_ID)

flows_client = globus_sdk.FlowsClient(app=my_app)

### Write Your Flow

Now that we have a compute function that:
- Runs the WPS
- Saves a file (returning the path)

We would like to transfer this back to the user, regardless of where they run it! We can use the Globus Transfer functionality, and combine these into a single flow. See the structure below.

In [4]:
flow_definition = {
    "Comment": "A Web Processing Services (WPS) Demonstration for ESGF Hosted Data",
    "StartAt": "TransferInput",
    "States": {
        "TransferInput": {
            "Comment": "Transfer input data",
            "Type": "Action",
            "ActionUrl": "https://transfer.actions.globus.org/transfer",
            "Parameters": {
                "source_endpoint.$": "$.input.source.id",
                "destination_endpoint.$": "$.input.destination.id",
                "DATA": [
                    {
                        "source_path.$": "$.input.source.path",
                        "destination_path.$": "$.input.destination.path",
                        "recursive": True,
                    }
                ]
            },
            "ResultPath": "$.TransferFiles",
            "WaitTime": 300,
            "Next": "ProcessESGFwps"
        },
        "ProcessESGFwps": {
            "Comment": "Process Web Processing Services for ESGF",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/",
            "Parameters": {
                "endpoint.$": "$.input.compute_endpoint",
                "function.$": "$.input.esgf_function",
                "kwargs.$": "$.input.esgf_kwargs"
            },
            "ResultPath": "$.ESGF_output",
            "WaitTime": 600,
            "End": True
        },
    }
}

#### Create the Flow with the Client
Now that we have our flow client and definition, we can create our flow!

In [5]:
flow = flows_client.create_flow(title="ESGF Flow Local", definition=flow_definition, input_schema={})


Please authenticate with Globus here:
-------------------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=3ec14a98-4bb5-492d-b71a-697960482d57&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+https%3A%2F%2Fauth.globus.org%2Fscopes%2Feec9b274-0c81-4334-bdc2-54e90e689b9a%2Fall&state=_default&response_type=code&code_challenge=4SI6HQFCtDugolQdyWQsV2IYjYKp5ETqHHZ6ai87pfU&code_challenge_method=S256&access_type=online&prefill_named_grant=esgf-wps-app+on+Maxs-MacBook-Pro-8
-------------------------------------



Enter the resulting Authorization Code here:  hVzPc8npiDB8soid7JcJobAlaqVNl7


In [6]:
flow_id = flow['id']
flow_id

'b58a9128-ca24-40a7-84ad-3b9570dff053'

#### Specify the Input
We need to specify the input for our flow - these variables are represented using the "$" syntax

In [14]:
flow_input = {
    "input": {
        "source": {
            "id": "03e6a23b-fb93-11ef-985b-0207be7ee3a1",
            "path": esgf_wps_data["odir"]
        },
        "destination": {
            "id": "03e6a23b-fb93-11ef-985b-0207be7ee3a1",
            "path": "/Users/mgrover/git_repos/esgf-crocus-globus-flows/flows/esgf/output"
        },
        "compute_endpoint": compute_endpoint,
        "esgf_kwargs": esgf_wps_data,
        "esgf_function": function_id,
    }
}

In [15]:
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    app=my_app,
)

In [19]:
run = specific_flow_client.run_flow(
  body=flow_input,
  label="ESGF WPS Example",
  tags=['ESGF', 'WPS', 'local']
)

In [20]:
# Get run details
# run = flows_client.get_run(run_id)

run_id = run['run_id']
run_status = run['status']
print("This flow can be monitored in the Web App:")
print(f"https://app.globus.org/runs/{run_id}")
print(f"Flow run started with ID: {run_id} - Status: {run_status}")

# Poll the Flow ser/vice to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')
    
# Run completed
print(json.dumps(run.data, indent=2))

This flow can be monitored in the Web App:
https://app.globus.org/runs/192fde4b-1c00-4c8a-9df5-74f34c343af5
Flow run started with ID: 192fde4b-1c00-4c8a-9df5-74f34c343af5 - Status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: ACTIVE
Run status: SUCCEEDED
{
  "run_id": "192fde4b-1c00-4c8a-9df5-74f34c343af5",
  "action_id": "192fde4b-1c00-4c8a-9df5-74f34c343af5",
  "flow_id": "b58a9128-ca24-40a7-84ad-3b9570dff053",
  "flow_title": "ESGF Flow Local",
  "flow_last_updated": "2025-04-02T17:26:03.562890+00:00",
  "start_time": "2025-04-02T17:27:44.437338+00:00",
  "completion_time": "2025-04-02T17:28:31.819000+00:00",
  "status": "SUCCEEDED",
  "display_status": "SUCCEEDED",
  "details": {
    "code": "FlowSucceeded",
    "output": {
      "input": {
        "source": {
          "id": "03e6a23b-fb93-11ef-985b-0207be7ee3a1",
          "path": "/Users/mgrover/git_repos